In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

# Leyendo lista de Colegios y Estudiantes

In [4]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [5]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [6]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [8]:
print("Archivos de estudiantes: ", estuDic['20131'],estuDic['20132'])
print("Archivos de colegios: ",coleDic['2013'])

Archivos de estudiantes:  SB11_20131.txt SB11_20132.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2013.txt


In [9]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20131'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')
df2 = spark.read.load(base2 + '/' + estuDic['20132'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [10]:
df1.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENETELEVISOR',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TIENE_NEVERA',
 'FAMI_TIENE_CELULAR',
 'FAMI_TELEFONO',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'ESTU_ANTECEDENTES',
 'ESTU_ANOSPREESCOLAR',
 'ESTU_ANOMATRICULAPRIMERO',
 'ESTU_ANOTERMINOQUINTO',
 'ESTU_ANOMATRICULASEXTO',
 'ES

In [12]:
df2.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENETELEVISOR',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TIENE_NEVERA',
 'FAMI_TIENE_CELULAR',
 'FAMI_TELEFONO',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'ESTU_ANTECEDENTES',
 'ESTU_ANOSPREESCOLAR',
 'ESTU_ANOMATRICULAPRIMERO',
 'ESTU_ANOTERMINOQUINTO',
 'ESTU_ANOMATRICULASEXTO',
 'ES

In [13]:
columnasPuntos20131 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_FISICA',
 'PUNT_INTERDISC_VIOLENCIAYSOC',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_PROFUNDIZA_MATEMATICA']

columnasPuntos20132 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_PROFUNDIZA_MATEMATICA',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_FISICA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_INTERDISC_VIOLENCIAYSOC'  
]

In [17]:
len(columnasPuntos20131)

14

In [16]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20131'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos20132)
df2 = spark.read.load(base2 + '/' + estuDic['20132'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos20132)

In [18]:

cols = columnasPuntos20132
cols.remove('COLE_COD_DANE_ESTABLECIMIENTO')

In [19]:
# SE unen los dos dataframes
df = unionAll(df1,df2)
for cp in cols:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    df = df.na.fill({cp:0})

In [20]:
print(df.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: long (nullable = true)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_PROFUNDIZA_MATEMATICA: float (nullable = false)
 |-- PUNT_CIENCIAS_SOCIALES: float (nullable = false)
 |-- PUNT_PROFUNDIZA_CSOCIALES: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)
 |-- PUNT_BIOLOGIA: float (nullable = false)
 |-- PUNT_FILOSOFIA: float (nullable = false)
 |-- PUNT_PROFUNDIZA_BIOLOGIA: float (nullable = false)
 |-- PUNT_FISICA: float (nullable = false)
 |-- PUNT_INTERDISC_MEDIOAMBIENTE: float (nullable = false)
 |-- PUNT_QUIMICA: float (nullable = false)
 |-- PUNT_LENGUAJE: float (nullable = false)
 |-- PUNT_PROFUNDIZA_LENGUAJE: float (nullable = false)
 |-- PUNT_INTERDISC_VIOLENCIAYSOC: float (nullable = false)

None


In [21]:
df = df.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [22]:
print("Numero de filas ", df.count())

Numero de filas  576092


In [23]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)

In [24]:
dsCountEst.show(10,False)
# Numero de estudiantes por instituciòn

+-----------------------------+-----+
|COLE_COD_DANE_ESTABLECIMIENTO|count|
+-----------------------------+-----+
|311001105863                 |1285 |
|311001105391                 |1144 |
|105001000108                 |1103 |
|105001013340                 |725  |
|105615000236                 |708  |
|305001003963                 |661  |
|311001104786                 |658  |
|176001005813                 |618  |
|105154000301                 |607  |
|168001003591                 |551  |
+-----------------------------+-----+
only showing top 10 rows



In [25]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['PUNT_MATEMATICAS']+df['PUNT_INGLES']+df['PUNT_CIENCIAS_SOCIALES']+
                     df['PUNT_BIOLOGIA']+df['PUNT_FILOSOFIA']+df['PUNT_FISICA']+df['PUNT_QUIMICA']+
                     df['PUNT_LENGUAJE']+df['PUNT_INTERDISC_MEDIOAMBIENTE']+df['PUNT_INTERDISC_VIOLENCIAYSOC']+
                     df['PUNT_PROFUNDIZA_BIOLOGIA']+
                     df['PUNT_PROFUNDIZA_CSOCIALES']+df['PUNT_PROFUNDIZA_LENGUAJE']+df['PUNT_PROFUNDIZA_MATEMATICA'])


## Resultados finales por instituciòn

In [26]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = df.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').mean()
df = df.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df = df.orderBy('rank',ascending= True)

In [27]:
df.select('COLE_COD_DANE_ESTABLECIMIENTO','avg(Suma)','rank').show(10,False)

+-----------------------------+-----------------+----+
|COLE_COD_DANE_ESTABLECIMIENTO|avg(Suma)        |rank|
+-----------------------------+-----------------+----+
|376001013441                 |601.4285714285714|1   |
|311001089221                 |597.0588235294117|2   |
|311848000278                 |586.6063829787234|3   |
|305001015503                 |584.0            |4   |
|368001000516                 |583.7954545454545|5   |
|368276000826                 |580.76           |6   |
|405440000789                 |580.3214285714286|7   |
|311769000165                 |579.0545454545454|8   |
|305001010781                 |575.6031746031746|9   |
|311001076634                 |574.7166666666667|10  |
+-----------------------------+-----------------+----+
only showing top 10 rows



## Colegios

In [28]:
# SE crea Dataframe de colegios en este año desaparecio la columna COLE_CODMPIO_COLEGIO
dfc = spark.read.load(base + '/' + coleDic['2013'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))

In [29]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- CLPL_N: integer (nullab

In [30]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [31]:
dfc = dfc.select(colsColegio)

In [32]:
dfc.show(5,False)

+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|120717             |7                     |7           |7          |6            |7             |6               |7            |6          |null          |null         |
|6833               |7                     |6           |7          |5            |7             |7               |6            |7          |null          |null         |
|6817               |8                     |7           |7          |7            |7             |7               |8            |7          |null

In [33]:
print("El numero de colegios es: ", dfc.select('COLE_CODIGO_COLEGIO').count())

El numero de colegios es:  8139


## Anàlisis comparativo entre estudiantes y colegios

In [34]:
dfjoin = df1.join(dfc, df1.COLE_COD_DANE_ESTABLECIMIENTO == dfc.COLE_CODIGO_COLEGIO)

In [35]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe còdigo dane en el archivo de colegios.